## Convert PBD file to Zarr file using `fastpdb`

In [1]:
import pandas as pd
import os
import numpy as np
import zarr
import pdb
from biotite.structure import Atom
from pathlib import Path

In [2]:
import fastpdb

in_file = fastpdb.PDBFile.read("8tav.pdb")
atom_array = in_file.get_structure(
    model=1, include_bonds=True, extra_fields=["atom_id", "b_factor", "occupancy", "charge"]
)

out_file = fastpdb.PDBFile()
out_file.set_structure(atom_array)
out_file.write("8tav_fastpdb.pdb")

In [3]:
atom_array

array([
	Atom(np.array([-21.721,  -0.403,  20.938], dtype=float32), chain_id="A", res_id=5, ins_code="", res_name="LEU", hetero=False, atom_name="N", element="N", atom_id=1, b_factor=41.27, occupancy=1.0, charge=0),
	Atom(np.array([-21.853,  -1.782,  20.457], dtype=float32), chain_id="A", res_id=5, ins_code="", res_name="LEU", hetero=False, atom_name="CA", element="C", atom_id=2, b_factor=39.9, occupancy=1.0, charge=0),
	Atom(np.array([-22.355,  -1.761,  19.009], dtype=float32), chain_id="A", res_id=5, ins_code="", res_name="LEU", hetero=False, atom_name="C", element="C", atom_id=3, b_factor=25.53, occupancy=1.0, charge=0),
	Atom(np.array([-23.016,  -0.809,  18.59 ], dtype=float32), chain_id="A", res_id=5, ins_code="", res_name="LEU", hetero=False, atom_name="O", element="O", atom_id=4, b_factor=29.11, occupancy=1.0, charge=0),
	Atom(np.array([-22.826,  -2.545,  21.351], dtype=float32), chain_id="A", res_id=5, ins_code="", res_name="LEU", hetero=False, atom_name="CB", element="C", atom

In [4]:
reloaded = fastpdb.PDBFile.read("8tav_fastpdb.pdb")

In [5]:
Path("8tav_fastpdb.pdb").stem

'8tav_fastpdb'

In [16]:
# convert atom arrays to dictionary
KEYS = [
    "X",
    "Y",
    "Z",
    "chain_id",
    "res_id",
    "ins_code",
    "res_name",
    "hetero",
    "atom_name",
    "element",
    "atom_id",
    "b_factor",
    "occupancy",
    "charge",
]


def pdb_to_dict(atom_array):
    pdb_dict = {k: [] for k in KEYS}
    for row in atom_array:
        pdb_dict["X"].append(row.coord[0])
        pdb_dict["Y"].append(row.coord[1])
        pdb_dict["Z"].append(row.coord[2])
        pdb_dict["chain_id"].append(row.chain_id)
        pdb_dict["res_id"].append(row.res_id)
        pdb_dict["ins_code"].append(row.ins_code)
        pdb_dict["res_name"].append(row.res_name)
        pdb_dict["hetero"].append(row.hetero)
        pdb_dict["atom_name"].append(row.atom_name)
        pdb_dict["element"].append(row.element)
        pdb_dict["atom_id"].append(row.atom_id)
        pdb_dict["b_factor"].append(row.b_factor)
        pdb_dict["occupancy"].append(row.occupancy)
        pdb_dict["charge"].append(row.charge)
    return pdb_dict

In [9]:
import zarr
import numpy as np

DTYPE_DICT = {
    "chain_id": "U4",
    "res_id": int,
    "ins_code": "U1",
    "res_name": "U5",
    "hetero": bool,
    "atom_name": "U6",
    "element": "U2",
}


def create_zarr_from_pdb(pdb_file, zarr_file):
    # load structure
    in_file = fastpdb.PDBFile.read(pdb_file)
    pdb_name = Path(pdb_file).stem
    atom_array = in_file.get_structure(
        model=1, include_bonds=True, extra_fields=["atom_id", "b_factor", "occupancy", "charge"]
    )

    # fastpbd to dict
    pdb_dict = pdb_to_dict(atom_array)

    # Create a Zarr store
    store = zarr.DirectoryStore(zarr_file)

    # Create a root group
    root = zarr.group(store=store)

    # Create group and add datasets
    group = root.create_group(pdb_name)
    for col_name, col_val in pdb_dict.items():
        col_val = np.array(col_val)
        # get the accepted dtype
        dtype = DTYPE_DICT.get(col_name, col_val.dtype)
        group.create_dataset(col_name, data=col_val, dtype=dtype)

In [10]:
!rm -r /Users/lu.zhu/Documents/Codebase/ValenceLab/polaris/notebook/dev/8tav_fastpbd.zarr
create_zarr_from_pdb("8tav.pdb", "8tav_fastpbd.zarr")

In [18]:
# load nested zarr file to dictionary
def load_group_as_numpy_arrays(group):
    """Load group datasets to numpy arrays exluding `column_names`"""
    arrays = {}
    for key, item in group.items():
        if isinstance(item, zarr.core.Array):
            arrays[key] = item[:]
        elif isinstance(item, zarr.hierarchy.Group):
            arrays[key] = load_group_as_numpy_arrays(item)
    return arrays

In [20]:
## load from zarr  and export to PDB
import biotite.structure as struc


def zarr_to_pdb(zarr_file, pdb_file, pdb_pointer):
    store = zarr.DirectoryStore(zarr_file)
    root = zarr.open_group(store=store, mode="r")  # 'r' mode for read-only
    group = root[pdb_pointer]
    atom_array = []
    atom_dict = load_group_as_numpy_arrays(group)

    # convert dictionary to array list of Atom object
    array_length = atom_dict["X"].shape[0]
    for ind in range(array_length):
        atom = Atom(
            coord=(atom_dict["X"][ind], atom_dict["Y"][ind], atom_dict["Z"][ind]),
            chain_id=atom_dict["chain_id"][ind],
            res_id=atom_dict["res_id"][ind],
            ins_code=atom_dict["ins_code"][ind],
            res_name=atom_dict["res_name"][ind],
            hetero=atom_dict["hetero"][ind],
            atom_name=atom_dict["atom_name"][ind],
            element=atom_dict["element"][ind],
            b_factor=atom_dict["b_factor"][ind],
            occupancy=atom_dict["occupancy"][ind],
            charge=atom_dict["charge"][ind],
        )
        atom_array.append(atom)

    # write the arrays to fastpdb file
    out_file = fastpdb.PDBFile()
    out_file.set_structure(struc.array(atom_array))
    out_file.write(pdb_file)

In [21]:
zarr_to_pdb("8tav_fastpbd.zarr", "8tav_from_fastpdb_zarr.pdb", pdb_pointer="8tav")

In [22]:
# Compare the structures
from pdb_convertor import compare_structures

are_identical = compare_structures("8tav_fastpdb.pdb", "8tav_from_fastpdb_zarr.pdb")

# Print the result
if are_identical:
    print("The structures are identical.")
else:
    print("The structures are not identical.")

RMSD: 0.0000 Å
The structures are identical.
